# Memory

## Memory Profiling 

### Standard Lib

In [32]:
import sys

In [33]:
sys.getsizeof(3)

28

In [34]:
sys.getsizeof(20.3)

24

In [35]:
sys.getrefcount(3)

487

In [36]:
int('1' + '0'*1000)

1000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [37]:
L = list(range(int(1e6)))

In [38]:
len(L)

1000000

In [39]:
sys.getsizeof(L)

9000112

In [40]:
sys.getsizeof([])

64

In [41]:
sys.getsizeof(L) / (1024**2)

8.583175659179688

In [42]:
int(1e6)* 28

28000000

In [43]:
sum(sys.getsizeof(x) for x in L)

27999996

### Pympler

In [44]:
m = tracker.SummaryTracker()

NameError: name 'tracker' is not defined

In [28]:
m.print_diff()

NameError: name 'm' is not defined

In [29]:
big = list(range(1_000_000))

In [30]:
m.print_diff()

NameError: name 'm' is not defined

In [ ]:
cd ../source/optimizing/

In [ ]:
cd measuring/


In [ ]:
# %load memory_size_pympler.py
# file: memory_size_pympler.py

"""Measure the size of used memory with a decorator.
"""

from __future__ import print_function

import functools                                                #1
import sys

if sys.version_info.major < 3:
    range = xrange

from pympler import tracker                                     #2

memory = {}                                                     #3


def measure_memory(function):                                   #4
    """Decorator to measure memory size.
    """

    @functools.wraps(function)                                  #5
    def _measure_memory(*args, **kwargs):                       #6
        """This replaces the function that is to be measured.
        """
        measurer = tracker.SummaryTracker()                     #
        for _ in range(2):                                      #8
            measurer.diff()                                     #9
        try:
            res = function(*args, **kwargs)                     #10
            return res
        finally:                                                #11
            memory[function.__name__] = (measurer.diff())
    return _measure_memory                                      #12


if __name__ == '__main__':

    @measure_memory                                             #13
    def make_big(number):
        """Example function that makes a large list.
        """
        return list(range(number))                              #14

    make_big(int(1e6))                                          #15
    print('used memory', memory)                                #16


In [ ]:
# %load memory_growth_pympler.py
# file memory_growth_pympler.py

"""Measure the memory growth during a function call.
"""
from __future__ import print_function

import sys

if sys.version_info.major < 3:
    range = xrange

from pympler import tracker                                     #1


def check_memory_growth(function, *args, **kwargs):             #2
    """Measure the memory usage of `function`.
    """
    measurer = tracker.SummaryTracker()                         #3
    for _ in range(2):                                          #4
        measurer.diff()                                         #5
    function(*args, **kwargs)                                   #6
    return measurer.diff()                                      #7

if __name__ == '__main__':

    def test():
        """Do some tests with different memory usage patterns.
        """

        def make_big(number):                                   #8
            """Function without side effects.

            It cleans up all used memory after it returns.
            """
            return list(range(number))

        data = []                                               #9

        def grow(number):
            """Function with side effects on global list.
            """
            for x in range(number):
                data.append(x)                                  #10

        size = int(1e6)
        print('memory make_big:', check_memory_growth(make_big,
                                                      size))     #11
        print('memory grow:', check_memory_growth(grow, size))   #12

    test()


In [ ]:
# %load pympler_list_growth.py
# file: pympler_list_growth.py

"""Measure the size of a list as it grows.
"""
from __future__ import print_function

import sys

from pympler.asizeof import asizeof, flatsize


if sys.version_info.major < 3:
    range = xrange


def list_mem(length, size_func=flatsize):
    """Measure incremental memory increase of a growing list.
    """
    my_list = []
    mem = [size_func(my_list)]
    for elem in range(length):
        my_list.append(elem)
        mem.append(size_func(my_list))
    return mem


if __name__ == '__main__':

    def main():
        """Show plot or numbers.
        """
        SIZE = 1000
        SHOW = 20

        for func in [flatsize, asizeof, sys.getsizeof]:
            mem = list_mem(SIZE, size_func=func)
            try:
                from matplotlib import pylab
                pylab.plot(mem)
                pylab.show()
            except ImportError:
                print('matplotlib seems not be installed. Skipping the plot.')
                if SIZE > SHOW:
                    limit = SHOW // 2
                    print(mem[:limit],
                          '... skipping %d elements ...' % (SIZE - SHOW),
                          end='')
                    print(mem[-limit:])
                else:
                    print(mem)


In [ ]:
%matplotlib inline

In [ ]:
main()

In [ ]:
# %load list_alloc_steps.py
# file: list_alloc_steps.py

"""Measure the number of memory allocation steps for a list.
"""
from __future__ import print_function

import sys

if sys.version_info.major < 3:
    range = xrange

from pympler.asizeof import flatsize


def list_steps(lenght, size_func=sys.getsizeof):
    """Measure the number of memory alloaction steps for a list.
    """
    my_list = []
    steps = 0
    int_size = size_func(int())
    old_size = size_func(my_list)
    for elem in range(lenght):
        my_list.append(elem)
        new_size = sys.getsizeof(my_list)
        if new_size - old_size > int_size:
            steps += 1
        old_size = new_size
    return steps


if __name__ == '__main__':
    steps = [10, 100, 1000, 10000, int(1e5), int(1e6), int(1e7)]
    print('Using sys.getsizeof:')
    for size in steps:
        print('%10d: %3d' % (size, list_steps(size)))
    print('Using pympler.asizeof.flatsize:')
    for size in steps:
        print('%10d: %3d' % (size, list_steps(size, flatsize)))


#### Run this from command line

`Weird results in mac (Increments), due to memory compression algorithms`

### Exercice 4.1.4

In [ ]:
m = tracker.SummaryTracker()

In [ ]:
m.print_diff()

In [ ]:
# %load make_list_gen.py
#file: make_list_gen.py

from __future__ import print_function

import sys
import time

if sys.version_info.major < 3:
    range = xrange


def make_list(n):
    return [x * 10 for x in range(n)]


def make_gen(n):
    return (x * 10 for x in range(n))


def test():
    n = int(1e4)  # 1e5, 1e6, 1e7, 1e8
    list_ = make_list(n)
    del list_
    gen = make_gen(n)
    del gen
    time.sleep(1)

test()


In [ ]:
m.print_diff()

In [ ]:
# %load make_list_gen.py
#file: make_list_gen.py

from __future__ import print_function

import sys
import time

if sys.version_info.major < 3:
    range = xrange

def make_list(n):
    return [x * 10 for x in range(n)]

def make_gen(n):
    return (x * 10 for x in range(n))


def test():
    n = int(1e4)  # 1e4, 1e5, 1e6, 1e7, 1e8
    list_ = make_list(n)
    del list_
    gen = make_gen(n)
    del gen
    time.sleep(1)

#for x in [1e4]: 
#    test(x)
test()

In [ ]:
mem_list = measure_memory(make_list)

In [ ]:
mem_gen = measure_memory(make_gen)

In [ ]:
n = 10

In [ ]:
memory

In [ ]:
mem_list(n)

In [ ]:
mem_gen(n);

In [14]:
memory

NameError: name 'memory' is not defined

## Weak references

In [2]:
cd ../source/optimizing/weakref/

/Volumes/HDD-Data/PycharmProjects/AdvancedPython/source/optimizing/weakref


In [4]:
# %load weakref_list.py
"""Create a weak reference with a list.
"""

import weakref


class MyList(list):
    """My list is just inherits from `list`.
    Need this because built-in list does not work with weak refs.
    """
    pass


def test():
    """Simple test.
    """
    my_list = MyList([1, 2, 3])

    print('List:', my_list)
    print('Weak reference count:', weakref.getweakrefcount(my_list))
    weak_list = weakref.ref(my_list)
    print('Creating a weak reference.')
    print('Weak reference count:', weakref.getweakrefcount(my_list))
    print('Calling the weakref object', weak_list())
    print('Deleting the original list.')
    del my_list
    print('Weak reference count:', weak_list())


if __name__ == '__main__':

    test()


List: [1, 2, 3]
Weak reference count: 0
Creating a weak reference.
Weak reference count: 1
Calling the weakref object [1, 2, 3]
Deleting the original list.
Weak reference count: None


In [10]:
# %load weak_value_dict.py
"""Example of WeakValueDictionary for id to object translation.

Inspired by the Python documentation:
https://docs.python.org/3/library/weakref.html?highlight=weakref#example
"""

import weakref


class IDStore:
    """Storage of objects by ID.
    """

    def __init__(self):
        self._id2obj_dict = weakref.WeakValueDictionary()


    def remember(self, obj):
        """Remember the object in a weak value dictionary.
        """
        oid = id(obj)
        self._id2obj_dict[oid] = obj
        return oid


    def id2obj(self, oid):
        """Retrieve the object by its id.
        """
        return self._id2obj_dict[oid]


class MyClass:
    """Simple example class.
    """
    pass

def test():
    """Simple test.

    Shows that class instance disappears from dictionary when deleted
    but not explicitly removed from the weak value dictionary used to
    store the object.
    """

    id_store = IDStore()

    print('First instance')
    print('===============')
    my_inst = MyClass()
    print('object id:', id_store.remember(my_inst))
    print('Is retrieved object my object?:',
          id_store.id2obj(id(my_inst)) is my_inst)
    print('Show weak value dict:', dict(id_store._id2obj_dict))
    print('delete instance')
    del my_inst
    print('Show weak value dict:', dict(id_store._id2obj_dict))

    print()
    print('Second instance')
    print('===============')
    my_inst = MyClass()  # catch freed id to avoid the same id in store
    my_inst = MyClass()
    print('object id:', id_store.remember(my_inst))
    print('Is retrieved object my object?:',
          id_store.id2obj(id(my_inst)) is my_inst)
    print('Show weak value dict:', dict(id_store._id2obj_dict))
    print('delete instance')
    del my_inst
    print('Show weak value dict:', dict(id_store._id2obj_dict))


if __name__ == '__main__':

    test()


First instance
object id: 4344152416
Is retrieved object my object?: True
Show weak value dict: {4344152416: <__main__.MyClass object at 0x102ee8160>}
delete instance
Show weak value dict: {}

Second instance
object id: 4344152864
Is retrieved object my object?: True
Show weak value dict: {4344152864: <__main__.MyClass object at 0x102ee8320>}
delete instance
Show weak value dict: {}


In [12]:
# %load supported_objects.py
"""Simple test for weakly reference-able objects.
"""

from collections import defaultdict
import weakref
import sys

class MyClass:
    """Empty class.
    """
    pass


def test(objects):
    """Show if objects can be weakly referenced.
    """
    for obj in objects:
        try:
            weakref.ref(obj)
            print('works for', type(obj))
        except TypeError:
            print("doesn't work for", type(obj))

if __name__ == '__main__':

    objects = [MyClass(), set(), frozenset(), sys, defaultdict(), [], {}, (),
               'a', 2, 3.5,  object(), range(10), zip()]
    test(objects)

works for <class '__main__.MyClass'>
works for <class 'set'>
works for <class 'frozenset'>
works for <class 'module'>
doesn't work for <class 'collections.defaultdict'>
doesn't work for <class 'list'>
doesn't work for <class 'dict'>
doesn't work for <class 'tuple'>
doesn't work for <class 'str'>
doesn't work for <class 'int'>
doesn't work for <class 'float'>
doesn't work for <class 'object'>
doesn't work for <class 'range'>
doesn't work for <class 'zip'>


### Exercice 4.2.4 

In [31]:
import weakref

class MyClass(object):
    pass

wd = weakref.WeakKeyDictionary()

my_inst1 = MyClass()
my_inst2 = MyClass()
wd[my_inst1] = 1
wd[my_inst2] = 2
print(dict(wd))
del my_inst1
print(dict(wd))

{<__main__.MyClass object at 0x102ee8d68>: 1, <__main__.MyClass object at 0x102ee8cf8>: 2}
{<__main__.MyClass object at 0x102ee8cf8>: 2}
